## 第一步：环境配置与数据读取

In [2]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
# import open3d as o3d  # 推荐安装 open3d 用于3D可视化

# 配置路径
DATA_ROOT = "/home/xzy/datasets/Rellis-3D" 
SEQ = "00004"
FRAME_ID = "000000"  # 示例帧

# 路径拼接 (根据 Rellis 目录结构)
# Rellis 通常结构: 00004/os1_cloud_node_kitti_bin/000000.bin
bin_path = os.path.join(DATA_ROOT, SEQ, 'os1_cloud_node_kitti_bin', f'{FRAME_ID}.bin')
img_path = os.path.join(DATA_ROOT, SEQ, 'pylon_camera_node', f'frame{FRAME_ID}-...jpg') # 需补全实际文件名

# 读取点云 (x, y, z, intensity)
points = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)
print(f"Point cloud shape: {points.shape}")

Point cloud shape: (131072, 4)


## 第二步：理解相机与雷达的关系（外参标定）

## 第三步：解析 poses.txt 与世界坐标系

坐标轴位姿可视化：红绿蓝 x y z

使用 o3d.geometry.TriangleMesh.create_sphere 在轨迹两端放置了球体。

起点：绿色 [0, 1, 0]，代表 "Go/Start"。

终点：红色 [1, 0, 0]，代表 "Stop/End"。

poses.txt立即为雷达系位姿（前x左y上z），可视化发现车子是倒开。

In [8]:
import open3d as o3d
import numpy as np



def parse_poses(filename):
    poses = []
    with open(filename, 'r') as f:
        for line in f:
            values = [float(v) for v in line.strip().split()]
            pose = np.eye(4)
            pose[:3, :4] = np.array(values).reshape(3, 4)
            poses.append(pose)
    return poses

pose_file = os.path.join(DATA_ROOT, SEQ, 'poses.txt')
poses = parse_poses(pose_file)

def visualize_trajectory_open3d(poses, visual_interval=10):
    """
    poses: 4x4 变换矩阵列表
    visual_interval: 每隔多少帧画一个坐标轴 (避免太密集)
    """
    geometries = []

    # 1. 创建轨迹线 (LineSet)
    # 提取所有平移向量 (位置)
    points = [p[:3, 3] for p in poses]
    
    # 定义线段连接关系: 0-1, 1-2, 2-3 ...
    lines = [[i, i+1] for i in range(len(points)-1)]
    
    # 创建 LineSet 对象
    line_set = o3d.geometry.LineSet()
    line_set.points = o3d.utility.Vector3dVector(points)
    line_set.lines = o3d.utility.Vector2iVector(lines)
    line_set.paint_uniform_color([0, 0, 0]) # 黑色轨迹线
    geometries.append(line_set)

    # 2. 创建坐标轴 (Coordinate Frames)
    # 红色=X(前), 绿色=Y(左), 蓝色=Z(上)
    for i in range(0, len(poses), visual_interval):
        # 创建一个小坐标轴，size 控制大小
        mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=2.0, origin=[0, 0, 0])
        
        # 将坐标轴变换到对应的位姿位置
        mesh_frame.transform(poses[i])
        geometries.append(mesh_frame)

    # 3. 特殊处理：起点 (Start Node)
    # 绿色球体 + 较大的坐标轴
    start_pose = poses[0]
    start_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.8)
    start_sphere.paint_uniform_color([0, 1, 0])  # 纯绿色
    start_sphere.translate(start_pose[:3, 3])
    
    start_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=3.0, origin=[0, 0, 0])
    start_frame.transform(start_pose)
    
    geometries.extend([start_sphere, start_frame])

    # 4. 特殊处理：终点 (End Node)
    # 红色球体 + 较大的坐标轴
    end_pose = poses[-1]
    end_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.8)
    end_sphere.paint_uniform_color([1, 0, 0])    # 纯红色
    end_sphere.translate(end_pose[:3, 3])
    
    end_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=3.0, origin=[0, 0, 0])
    end_frame.transform(end_pose)
    
    geometries.extend([end_sphere, end_frame])

    # 5. 启动可视化
    print("Green sphere: START | Red sphere: END")
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name="Trajectory Viewer (Start=Green, End=Red)", width=1280, height=720)
    
    for g in geometries:
        vis.add_geometry(g)
    
    # 获取渲染控制并设置点的大小（如果需要的话）
    opt = vis.get_render_option()
    opt.background_color = np.asarray([0.1, 0.1, 0.1]) # 设置深色背景更易观察坐标轴
    opt.line_width = 2.0
    
    vis.run()
    vis.destroy_window()

visualize_trajectory_open3d(poses, visual_interval=10)

Green sphere: START | Red sphere: END


## 第四步：点云建图（Point Cloud Mapping）

In [13]:
import os
import numpy as np
import open3d as o3d

# ================= 1. 配置路径 =================
# 请根据实际情况修改
LIDAR_FOLDER = 'os1_cloud_node_kitti_bin' # Rellis 对应雷达数据的文件夹名

# ================= 2. 辅助函数 =================

def load_lidar_col(bin_path):
    """
    读取点云并转换为齐次坐标列向量
    Return: (4, N) numpy array
    """
    # 1. 读取原始数据 (N, 4): [x, y, z, intensity]
    points = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)
    
    # 2. 提取 XYZ (N, 3)
    xyz = points[:, :3]
    
    # 3. 构建齐次坐标 (N, 4): [x, y, z, 1]
    num_points = xyz.shape[0]
    ones = np.ones((num_points, 1))
    points_h = np.hstack([xyz, ones])
    
    # 4. 转置为列向量 (4, N) -> 每一列是一个点 p = [x, y, z, 1]^T
    return points_h.T

def parse_poses(filename):
    """
    读取 Kitti 格式的 poses.txt
    Return: list of 4x4 matrices
    """
    poses = []
    if not os.path.exists(filename):
        print(f"Error: {filename} not found!")
        return []
        
    with open(filename, 'r') as f:
        for line in f:
            values = [float(v) for v in line.strip().split()]
            pose = np.eye(4)
            pose[:3, :4] = np.array(values).reshape(3, 4)
            poses.append(pose)
    return poses

# ================= 3. 主逻辑：建图 =================

# 准备路径
pose_file = os.path.join(DATA_ROOT, SEQ, 'poses.txt')
lidar_dir = os.path.join(DATA_ROOT, SEQ, LIDAR_FOLDER)

# 读取所有位姿
poses = parse_poses(pose_file)
print(f"Total poses loaded: {len(poses)}")

# 存储世界坐标系下的点 (最终会转回 N,3 给 Open3D)
global_map_points_cols = [] 
# 存储用于可视化的位姿几何体
visual_geometries = []

# 参数设置
STEP = 10  # 每隔 10 帧取一次
MAX_FRAMES = 200 # 限制总帧数，方便快速测试 (如果显存够大可以设为 len(poses))

print("Start mapping...")

for i in range(0, min(len(poses), MAX_FRAMES), STEP):
    # --- A. 获取当前位姿 T (4, 4) ---
    # 理解：T 把点从雷达系转到世界系
    T_world_lidar = poses[i] 
    
    # --- B. 获取文件名并读取点云 P_local (4, N) ---
    # Rellis 文件名通常是 000000.bin, 000001.bin ... 需要对其索引补零
    file_idx = i 
    bin_filename = f"{file_idx:06d}.bin" 
    bin_path = os.path.join(lidar_dir, bin_filename)
    
    if not os.path.exists(bin_path):
        print(f"Skipping frame {i}, file not found: {bin_path}")
        continue

    # P_local 是 (4, N) 列向量
    P_local = load_lidar_col(bin_path)
    
    # --- C. 核心变换公式: P_world = T * P_local ---
    # (4, 4) @ (4, N) -> (4, N)
    P_world = T_world_lidar @ P_local
    
    # --- D. 存入列表 (只取前3行 XYZ) ---
    # 我们先存着列向量，最后再合并转置
    global_map_points_cols.append(P_world[:3, :]) # (3, N)

    # --- E. 创建位姿可视化标记 (Coordinate Frame) ---
    # 创建一个小坐标轴，红=x, 绿=y, 蓝=z
    axis_mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.5, origin=[0, 0, 0])
    # 将坐标轴变换到当前位姿的位置
    axis_mesh.transform(T_world_lidar)
    visual_geometries.append(axis_mesh)
    
    if i % 50 == 0:
        print(f"Processed frame {i}...")

# ================= 4. 数据合并与 Open3D 可视化 =================

if global_map_points_cols:
    print("Merging points...")
    # 1. 水平拼接所有 (3, N) 数组 -> (3, Total_N)
    all_points_col = np.hstack(global_map_points_cols)
    
    # 2. 转置回 (Total_N, 3) 以符合 Open3D 格式
    all_points_np = all_points_col.T
    
    print(f"Total points in map: {all_points_np.shape[0]}")
    
    # 3. 创建点云对象
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(all_points_np)
    
    # (可选) 降采样：如果点太多导致卡顿，可以开启这行
    pcd = pcd.voxel_down_sample(voxel_size=0.2) 
    
    # 4. 将点云加入可视化列表
    visual_geometries.append(pcd)
    
    print("Visualizing... (Close the window to exit)")
    
    # 5. 启动可视化窗口
    # 你会看到一个稠密的点云地图，以及其中穿插的一串红绿蓝坐标轴（代表轨迹）
    o3d.visualization.draw_geometries(visual_geometries, 
                                      window_name="Rellis-3D Mapping & Trajectory",
                                      width=1024, height=768,
                                      left=50, top=50)
else:
    print("No points loaded. Check paths.")

Total poses loaded: 2059
Start mapping...
Processed frame 0...
Processed frame 50...
Processed frame 100...
Processed frame 150...
Merging points...
Total points in map: 2621440
Visualizing... (Close the window to exit)
